In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
%matplotlib notebook

from scipy import signal

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objects as go
from scipy.ndimage import gaussian_filter1d

from plotly.subplots import make_subplots

init_notebook_mode(connected=True)

In [3]:
df = pd.read_csv('data/capture_mcu_input.csv', names=('raw_current', 'busvoltage', 'raw_shuntvoltage', 'power'))
print(df.head())

df_current = gaussian_filter1d(df['raw_current'] / 20, 3)
df_voltage = df['busvoltage'] / 1000

x = np.arange(len(df_current)) * 10

current_line = go.Scatter(x=x, y=df_current, name='I (mA)')
voltage_line = go.Scatter(x=x, y=df_voltage, name='Vbus (V)')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(current_line, secondary_y=False)
fig.add_trace(voltage_line, secondary_y=True)

fig.update_layout(title_text='V/I measured coming into MCU')
fig.update_xaxes(title_text='T (ms)')

fig.show()

   raw_current  busvoltage  raw_shuntvoltage  power
0          522        6604               502    331
1          480        6696               500    329
2          484        6748               425    280
3          520        6604               505    333
4          478        6696               499    329


In [4]:
df_sweep = pd.read_csv('data/capture_psu_sweep.csv', names=('raw_current', 'busvoltage', 'raw_shuntvoltage', 'power'))

df_current = df_sweep['raw_current'] / 20
df_voltage = df_sweep['busvoltage'] / 1000

# df_current = (df_current - min(df_current)) / (max(df_current) - min(df_current))
# df_voltage = (df_voltage - min(df_voltage)) / (max(df_voltage) - min(df_voltage))

x = np.arange(len(df_current)) * 0.010

current_line = go.Scatter(x=x, y=df_current, name='I (mA)')
voltage_line = go.Scatter(x=x, y=df_voltage, name='Vbus (V)')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(current_line, secondary_y=False)
fig.add_trace(voltage_line, secondary_y=True)

fig.update_layout(title_text='V/I measured coming into MCU using PSU sweep from 7-8V')
fig.update_xaxes(title_text='T (s)')

fig.show()

In [5]:
from pathlib import Path

data_path = Path('data/samples')
dfs = {}

for sample_file in data_path.glob('*'):
    filename = sample_file.stem.split('_')
    flowrate = float(filename[-2] + '.' + filename[-1])
    dfs[flowrate] = pd.read_csv(sample_file, names=('raw_current', 'busvoltage', 'raw_shuntvoltage', 'power'))

dfs = dict(sorted(dfs.items(), key=lambda d: d[0]))
print('flowrates:', dfs.keys())

flowrates: dict_keys([7.5, 7.8, 8.0, 8.3, 8.5, 8.7, 9.0, 9.3, 9.5, 9.8])


In [6]:
df_current = dfs[7.5]['raw_current'] / 20
df_voltage = dfs[7.5]['busvoltage'] / 1000

x = np.arange(len(df_current)) * 0.010

current_line = go.Scatter(x=x, y=df_current, name='I (mA)')
voltage_line = go.Scatter(x=x, y=df_voltage, name='Vbus (V)')

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(current_line, secondary_y=False)
fig.add_trace(voltage_line, secondary_y=True)
fig.update_layout(title_text='V/I measured coming into cap at flowrate of 7.5 l/min')
fig.update_xaxes(title_text='T (s)')
fig.show()

In [7]:
fig = make_subplots()

x = np.arange(min(len(df) for df in dfs.values())) * 0.01

for flowrate, df in dfs.items():
    fig.add_trace(go.Scatter(x=x, y=df['busvoltage'] / 1000, name=f'Vbus ({flowrate} l/min)'))
    
fig.show()

In [8]:
fig = make_subplots()

for flowrate, df in dfs.items():
    fig.add_trace(go.Scatter(x=x, y=df['raw_current'] / 20, name=f'I ({flowrate} l/min)'))
    
fig.show()

In [28]:
from collections import defaultdict

features = defaultdict(list)

for fr, df in dfs.items():
    
    voltage = df['busvoltage'] / 1000
    current = df['raw_current'] / 20
    
    features['flowrate'].append(fr)
    features['V_max'].append(voltage.max())
    features['V_min'].append(voltage.min())
    features['V_mean'].append(voltage.mean())
    features['V_midrange'].append(voltage.min() + (voltage.max() - voltage.min()) / 2)
    features['I_mean'].append(current.mean())
    
df_features = pd.DataFrame(data=features)
df_features.head()

,flowrate,V_max,V_min,V_mean,V_midrange,I_mean
0,7.5,7.596,7.248,7.436121,7.422,40.159662
1,7.8,7.860,7.348,7.608788,7.604,42.833825
2,8.0,8.184,7.816,8.006633,8.000,46.030424
3,8.3,8.444,8.060,8.243769,8.252,48.060964
4,8.5,8.648,8.320,8.485698,8.484,50.906167


In [51]:
import scipy

def curve(x, a, b):
    return a * x + b

popt, _ = scipy.optimize.curve_fit(curve, features['flowrate'], features['V_mean'])
print(popt)

fig = make_subplots()
fig.add_trace(go.Scatter(x=features['flowrate'], y=features['V_mean'], name='Vmean'))
fig.add_trace(go.Scatter(x=features['flowrate'], y=features['V_max'], name='Vmax'))
fig.add_trace(go.Scatter(x=features['flowrate'], y=features['V_min'], name='Vmin'))

# flowrates_curve = np.linspace(7.5, 10, 1000)
# fig.add_trace(go.Scatter(x=flowrates_curve, y=curve(flowrates_curve, *popt), name='Curve'))

fig.show()

[ 1.09329187 -0.80758944]
